# Kaggle M5 Prediction Competition Team Terp Report
<img src="https://globalbiodefense.com/wp-content/uploads/2014/06/umd_robert_h_smith.jpg" width="600px">
<img src="https://cdn.mos.cms.futurecdn.net/nQmYiDoMBoTSewCLEhCouU-1024-80.jpg.webp" width="400px">

## Before You Start Reading
It was such a rewardful and exciting journey to participate in the 2020 M5 accuracy competition! Although our team merely ranked at 721st place among 5558 teams, we learned and grew as data analysts. 

This journey allowed us to spend time with new concepts and tools such as time-series data and panel data, LSTM, GRU, Facebook Prophet, and Keras. 

We are not experts in data science, but we would love to be one in the future. If you have any comments or advice, please don't hesitate to let us know in the comment or reach us out on LinkedIn. 

Thank you for your time, and please enjoy it! 

## Team Member:
1. [Hao Jiang](http://www.linkedin.com/in/umdhaojiang/)
2. [Shruthi Suresh](http://www.linkedin.com/in/shruthi-suresh-213157ab/)
3. [Alexander Binder](http://www.linkedin.com/in/alexander-jm-binder/)

Special Thanks to Professor [Liye Ma](http://www.linkedin.com/in/liye-ma-7b10492/) for his guildence and advise along the way

# Content of The Report

* [<font size=4>Description</font>](#1)
    * [Competition Description](#1.1)
    * [Variable Dictionary](#1.2)

* [<font size=4>Evaluation & Project Objective</font>](#2)
    * [Submission File Evaluation](#2.1)
    * [Project Objective](#2.2)
    
* [<font size=4>Python Package Used</font>](#3)

* [<font size=4>Initial Data Set Exploration</font>](#4)
    * [Calendar](#4.1)
    * [Sales_Train_Validation](#4.2)
    * [Sell_Price](#4.3)

* [<font size=4>Data Cleaning and Visualization</font>](#5)
    * [Data Cleaning](#5.1)
    * [Visualization](#5.2)

* [<font size=4>Prediction Methodology 1 - Time Series With Facebook Prophet</font>](#6)
    * [Data Merging](#6.1)
    * [Prediction](#6.2)

* [<font size=4>Prediction Methodology 2 - Time Series With LSTM</font>](#7)
    * [Data Merging](#7.1)
    * [Prediction](#7.2)

# Description <a id="1"></a>

## Competition Description <a id="1.1"></a>
In this competition, participants will use hierarchical sales data from Walmart, the world’s largest company by revenue, to **forecast daily sales for the next 28 days**. The data, covers stores in three US States (California, Texas, and Wisconsin) and includes item level, department, product categories, and store details. In addition, it has explanatory variables such as price, promotions, day of the week, and special events. Together, this robust dataset can be used to improve forecasting accuracy.

## Variable Dictionary <a id="1.2"></a>

[Click to Exam the Datasets](http://www.kaggle.com/c/m5-forecasting-accuracy/data)

**File 1: Calendar.csv:**

| Variable                      	| Explanation                                                                      	|
|-------------------------------	|----------------------------------------------------------------------------------	|
| date                          	| The date in a “y-m-d” format.                                                    	|
| wm_yr_wk                      	| The id of the week the date belongs to.                                          	|
| weekday                       	| The type of the day (Saturday, Sunday, …, Friday).                               	|
| wday                          	| The id of the weekday, starting from Saturday.                                   	|
| month                         	| The month of the date.                                                           	|
| year                          	| The year of the date.                                                            	|
| event_name_1                  	| If the date includes an event, the name of this event.                           	|
| event_type_1                  	| If the date includes an event, the type of this event.                           	|
| event_name_2                  	| If the date includes a second event, the name of this event.                     	|
| event_type_2                  	| If the date includes a second event, the type of this event.                     	|
| snap_CA, snap_TX, and snap_WI 	| A binary variable, whether the stores allow SNAP purchases on the examined date. 	

**File 2: Sell_Prices.csv**

| Variable   	| Explanation                                                                                                    	|
|------------	|----------------------------------------------------------------------------------------------------------------	|
| store_id   	| The id of the store where the product is sold.                                                                 	|
| item_id    	| The id of the product.                                                                                         	|
| wm_yr_wk   	| The id of the week.                                                                                            	|
| sell_price 	| The price of the product for the given week/store. The price is provided per week (average across seven days). 	|

**File 3: sales_train_validation**

| Variable                   	| Explanation                                                  	|
|----------------------------	|--------------------------------------------------------------	|
| item_id                    	| The id of the product.                                       	|
| dept_id                    	| The id of the department the product belongs to.             	|
| cat_id                     	| The id of the category the product belongs to.               	|
| store_id                   	| The id of the store where the product is sold.               	|
| state_id                   	| The State where the store is located.                        	|
| d_1, d_2, …, d_i, … d_1941 	| The number of units sold at day i, starting from 2011-01-29. 	|

# Evaluation & Project Objective <a id="2"></a>

## Submission File Evaluation <a id="2.1"></a>
Each row contains an id that is a concatenation of an `item_id` and a `store_id`, which is either `validation` or `evaluation`. Participants are predicting 28 forecast days `(F1-F28)` of items sold for each row. For the `validation` rows, this corresponds to `d_1914 - d_1941`, and for the `evaluation` rows, this corresponds to `d_1942 - d_1969`.

**Submission File Example**

| ID                            	| F1 	| F2 	| F3 	| F4 	| F5 	| ...... 	| F27 	| F28 	|
|-------------------------------	|----	|----	|----	|----	|----	|--------	|-----	|-----	|
| HOBBIES_1_001_CA_1_validation 	| 1  	| 2  	| 3  	| 4  	| 5  	| ...... 	| 3   	| 1   	|
| HOBBIES_1_002_CA_1_validation 	| 3  	| 3  	| 9  	| 4  	| 2  	| ...... 	| 1   	| 3   	|
| ............................. 	| .. 	| .. 	| .. 	| .. 	| .. 	| ...... 	| ..  	| ..  	|
| HOBBIES_1_001_CA_1_evaluation 	| 3  	| 5  	| 9  	| 0  	| 3  	| ...... 	| 2   	| 2   	|
| HOBBIES_1_002_CA_1_evaluation 	| 2  	| 0  	| 0  	| 1  	| 0  	| ...... 	| 3   	| 4   	|

## Project Objective <a id="2.2"></a>
Use the information such as `date`, `sell price`, and `state ID` for all ID between **day 1 to day 1913** to predict the `unit sales` between **day 1914 - day 1941** (Validation Data) and unit sales between **day 1942 - day 1969** (Evaluation Data).

* `Independent Variable`: date, snap_CA, snap_WI, snap_TX, sell_price, etc 
* `Dependent Variable`: Unit Sales 

# Python Package Used <a id="3"></a>

In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.impute import SimpleImputer

import pystan
import fbprophet

from keras.models import Sequential
from keras.layers import Dense 
from keras.metrics import RootMeanSquaredError
from keras.layers import BatchNormalization
from keras.callbacks import EarlyStopping
from keras import backend as K 
from keras.layers import LSTM
from keras.layers import GRU

# Initial Data Set Exploration <a id="4"></a>

## Calendar <a id="4.1"></a>

**Takeaways from Calendar Dataset**
> - 1969 rows represent `day_1` to `day_1969`
> - 14 columns represent the characteristic for each days
> - `Date` from 2011-01-29 to 2016-06-19
> - The number of unique `weekday` & `wday` are similar
> - The number of unique `years` are similar except year 2016
> - Binary variables `snap_CA`, `snap_TX`, and `snap_WI` have roughly 33% `1` and 67% `0`. Not a good variable to include in prediction model. 
> - 4 variables (`event_name_1`,`event_type_1`, `event_name_2`, `event_type_2`) have missing values. And almost all values for these variables are missing

In [ ]:
calendar = pd.read_csv('../input/m5-forecasting-accuracy/calendar.csv',parse_dates=['date'])
calendar.head(3)

In [ ]:
calendar.info()

In [ ]:
print('Date Start: ' + str(min(calendar['date'])))
print('Date Start: ' + str(max(calendar['date'])))
print('\n')
print('# of Unique days')
print(calendar['weekday'].value_counts())
print('\n')
print('# of Unique year')
print(calendar['year'].value_counts())
print('\n')
print('# of NaN in event_name_1,event_type_1,event_name_2,event_type_2')
print('Total # of missing value in event_name_1: ' + str(len(calendar.loc[calendar['event_name_1'].isna(),'event_name_1'])))
print('Total # of missing value in event_type_1: ' + str(len(calendar.loc[calendar['event_type_1'].isna(),'event_type_1'])))
print('Total # of missing value in event_name_2: ' + str(len(calendar.loc[calendar['event_name_2'].isna(),'event_name_2'])))
print('Total # of missing value in event_type_2: ' + str(len(calendar.loc[calendar['event_type_2'].isna(),'event_type_2'])))
print('\n')
print('# of 0&1 for binary variable snap_CA')
print(calendar['snap_CA'].value_counts())
print('\n')
print('# of 0&1 for binary variable snap_TX')
print(calendar['snap_TX'].value_counts())
print('\n')
print('# of 0&1 for binary variable snap_WI')
print(calendar['snap_WI'].value_counts())

## sales_train_validation <a id="4.2"></a>

**Takeaways from sales_train_validation data set**
> - `30490 rows` represent `30490 unique IDs`
> - Columns such as `d_1`, `d_2`, `etc` represent the sales for each ID in each day
> - 5 columns(`item_id`, `dept_id`, `cat_id`, `state_id`, and `store_id`) represent the characteristic of each ID

> - The id is composed by `item_id + store_id + string validation or evaluation`
> - The dataset only includes `unit sales` from **day 1 to day 1913**
> - [The data is in wide data format](http://www.theanalysisfactor.com/wide-and-long-data/), **we have to turn it to long data format**
> - The Final submission file is composed by 30490 for validation and 30490 for evaluation. In total, the **Final Submission should have 60980 rows**

In [ ]:
sales_train_validation = pd.read_csv('../input/m5-forecasting-accuracy/sales_train_validation.csv')
sales_train_validation.head(3)

In [ ]:
sales_train_validation.info()

In [ ]:
print('# of unique ID' + str(len(sales_train_validation['id'].value_counts())))
print('\n')
print('# of Unique State')
print(sales_train_validation['state_id'].value_counts())
print('\n')
print('# of Unique category')
print(sales_train_validation['cat_id'].value_counts())
print('\n')
print('# of unique item: ' + str(len(sales_train_validation['item_id'].value_counts())))
print(sales_train_validation['cat_id'].value_counts())
print('\n')
print('# of Unique Store')
print(sales_train_validation['store_id'].value_counts())

## sell_price <a id="4.3"></a>

**Takeaways from sales_train_validation data set**
> - The dataset has `6841121 rows` and `4 columns` 
> - The maximium sell_price is much higher than the mean & median, and it suggests **the data contains outliers**

> - Each row represent each item's sell_price across all store in each week
> - The data includes sell price from at day 1 to day 1969 (all validation & eveluation data) 


In [ ]:
sell_price = pd.read_csv('../input/m5-forecasting-accuracy/sell_prices.csv')
sell_price.head(3)

In [ ]:
sell_price.info()

In [ ]:
print('Statistic for Variable sell_price')
print(round(sell_price['sell_price'].describe(),2))
print('\n')
print('Sell Price Start at week: ' + str(min(sell_price['wm_yr_wk'])))
print('Sell Price End at week: ' + str(max(sell_price['wm_yr_wk'])))
print('\n')
print('The Start Week corresponding day: ')
print(calendar.loc[calendar['wm_yr_wk'] == 11101,'d'])
print('\n')
print('The End Week corresponding day: ')
print(calendar.loc[calendar['wm_yr_wk'] == 11621,'d'])

# Data Cleaning & Visualization <a id="5"></a>

## Data Cleaning <a id="5.1"></a>

### Data Cleaning - Calendar

In [ ]:
### 1: Turn the np.NaN values in event_name_1, event_type_1, event_name_2,event_type_2 to 0. 
categorical_Imputer = SimpleImputer(missing_values=np.nan,strategy='constant',fill_value=0)
calendar.iloc[:,[7,8,9,10]] = categorical_Imputer.fit_transform(calendar.iloc[:,[7,8,9,10]])

### 2: Turn event_name_1, event_type_1, event_name_2,event_type_2 to binary variables
calendar['event_name_1'] = calendar['event_name_1'].astype(str)
calendar['event_type_1'] = calendar['event_type_1'].astype(str)
calendar['event_name_2'] = calendar['event_name_2'].astype(str)
calendar['event_type_2'] = calendar['event_type_2'].astype(str)
# Create Dummy Holder
calendar['new_event_name_1'] = 0
calendar['new_event_type_1'] = 0
calendar['new_event_name_2'] = 0
calendar['new_event_type_2'] = 0
# Create Dummy holder based on original variables
for i in range(0,len(calendar['event_name_1'])): 
  if calendar.iloc[i,7] != '0':
    calendar.iloc[i,14] = 1

for i in range(0,len(calendar['event_type_1'])): 
  if calendar.iloc[i,8] != '0':
    calendar.iloc[i,15] = 1

for i in range(0,len(calendar['event_type_1'])): 
  if calendar.iloc[i,9] != '0':
    calendar.iloc[i,16] = 1

for i in range(0,len(calendar['event_type_1'])): 
  if calendar.iloc[i,10] != '0':
    calendar.iloc[i,17] = 1

calendar = calendar.drop(labels=['event_name_1','event_type_1','event_name_2','event_type_2'],axis=1)

calendar = calendar.rename(columns={'new_event_name_1':'event_name_1',
                                    'new_event_type_1':'event_type_1',
                                    'new_event_name_2':'event_name_2',
                                    'new_event_type_2':'event_type_2'})

In [ ]:
### 3: Replace weekday to numerical values
dayOfWeek={'Monday':0, 'Tuesday':1, 'Wednesday':2,'Thursday':3,'Friday':4,'Saturday':5,'Sunday':6}
calendar = calendar.replace({'weekday':dayOfWeek})

### 4: Drop variables event_name_1, event_name_2, wday since it is not insightful 
calendar = calendar.drop(labels=['event_name_1','event_name_2','wday'],axis=1)

### 5: code the year variable to categorical variable to save the RAM size
calendar.loc[calendar['year']==2012,'year'] = 5 ### Count in this level: 11159340
calendar.loc[calendar['year']==2015,'year'] = 4 ### Count in this level: 11128850
calendar.loc[calendar['year']==2014,'year'] = 3 ### Count in this level: 11128850
calendar.loc[calendar['year']==2013,'year'] = 2 ### Count in this level: 11128850
calendar.loc[calendar['year']==2011,'year'] = 1 ### Count in this level: 10275130
calendar.loc[calendar['year']==2016,'year'] = 0 ### Count in this level:  3506350

### 6: Take out the 'd_' in each rows for variable 'd'
calendar['d'] = calendar['d'].map(lambda x: x.lstrip('d_'))
calendar['d'] = calendar['d'].astype('int64')

### 7: Save the cleaned dataset
calendar.to_csv('../input/m5-forecasting-accuracy/new_calendar.csv',index=False)

In [ ]:
### Calendar Dataset after Data Cleaning
calendar.head()

### Data Clearning - sales_train_validation

In [ ]:
### 1: Melt the data as mentioned before
sales_train_validation = sales_train_validation.melt(id_vars=['id','item_id','dept_id','cat_id','store_id','state_id'],var_name='d',value_name='Unit_Sales')

### 2: Get rid of d_ in d column
sales_train_validation['d'] = sales_train_validation['d'].map(lambda x: x.lstrip('d_'))
sales_train_validation['d'] = sales_train_validation['d'].astype('int64')

### 4: Recode cat_id & state_id to numerical expression
sales_train_validation.loc[sales_train_validation['cat_id']=='HOBBIES','cat_id'] = 0 
sales_train_validation.loc[sales_train_validation['cat_id']=='HOUSEHOLD','cat_id'] = 1
sales_train_validation.loc[sales_train_validation['cat_id']=='FOODS','cat_id'] = 2 

sales_train_validation.loc[sales_train_validation['state_id']=='TX','state_id'] = 0
sales_train_validation.loc[sales_train_validation['state_id']=='WI','state_id'] = 1 
sales_train_validation.loc[sales_train_validation['state_id']=='CA','state_id'] = 2 

In [ ]:
### sales_train_validation after data cleaning
sales_train_validation.head()

## Visualization <a id="5.2"></a>

**event_type_1 for each weekday & year**

In [ ]:
fig,(ax0,ax1) = plt.subplots(nrows=1,ncols=2,figsize=(20,10))
sns.countplot(x='weekday',data=calendar,hue = 'event_type_1',ax=ax0)
sns.countplot(x='year',data=calendar,hue = 'event_type_1',ax=ax1)

ax0.set_title('Which weekday has most event')
ax1.set_title('Which year has most event')
plt.show()

**KDE Plot for Sell Price**

In [ ]:
### KDE Plot
sns.distplot(sell_price['sell_price'],bins=2,hist=False,rug=True,norm_hist=True,kde_kws={'shade':True})
plt.title('KDE Plot of Distribution for Sell Price')
plt.ylabel('Density')
plt.show()

**EDCF Chart for Sell Price**
> - Almost 100% of the sales price are less than `$35` and there are a couple of outliers larger than `$40`

In [ ]:
### EDCF Chart: 
# Almost 100% of the sales price are less than $35. There are a couple of outliers larger than 40
x = np.sort(sell_price['sell_price'])
y = np.arange(1,len(x)+1) / len(x)

percentile = np.array([0,25,50,75,100])
x_percent = np.percentile(x,q=percentile)

plt.plot(x,y,marker='.',linestyle='none',color='orange')
plt.plot(x_percent,x_percent/100,marker='D',linestyle='none',color='green')
plt.title('EDCF Chart for Sell Price')
plt.xlabel('Percentage')
plt.ylabel('EDCF')
plt.legend(('sell_price','sell_price_percentile'))
plt.show()

**Pie Chart for Unit Sales by Category - Percentage**
<img src="http://drive.google.com/uc?export=view&id=1FK22rK-aoPUFrh0kcSzN4nb_h6_gzt96" width=600px>

**Bar Chart for Unit Sales by Category & States**
<img src="http://drive.google.com/uc?export=view&id=18jeZKnaGVJxzQDJbhicA9cWKdfJD9Yy3" width=600px>

**Unit Sales Trend by Days & Category (Between day 1 to day 1913)** 
> - Seasonality is strong and identical for all three categories. 

<img src="http://drive.google.com/uc?export=view&id=1TsllepyJiL-gDQkwT7Y6d5Y8VcG8Yqxs" width=600px>


**Unit Sales Trend by Days & States (Between day 1 to day 1913)** 
> - Seasonality is strong and identical for all three States
> - Wisconsin and Texas has almost identical sales

<img src="http://drive.google.com/uc?export=view&id=1HCxCxmy5VVtLir1m9uCsrajHcLgxR4Qw" width=600px>


# Prediction Methodology 1 - Time Series With Facebook Prophet <a id="6"></a>

## Date Merging for Facebook Prophet <a id="6.1"></a>
> - Facebook Prophet only accept two columns which are the Time Series & the Dependent variable (in this case, Unit Sales) 
> - In this case,The infomation we need can be found at calendar dataset & sales_train_validation dataset and the common variable for both datasets is `d`
> - For train dataset, we will use infomation from 2014 to aviod long running time

In [ ]:
### 1: Prepare the calendar & sales_train_validation datasets

# 1-1: Select `year` start from 2014
calendar = calendar[calendar['year'] > 2013]
# 1-2: Keep the date for FB prophet & d for data merging
calendar = calendar.loc[:,['date','d']]
# 1-3: Keep `id` for indication, `d` for data merging, and `Unit_Sales` for FB prophet prediction
sales = sales.loc[:,['id','d','Unit_Sales']]

### 2: Merge two datasets
Time_Series = sales.merge(right=calendar,how='inner',on='d')

### 3: Drop the Unnessasary Columns
Time_Series.drop(labels='d',axis=1,inplace=True)

### 4: Extract the id info to seperate the whole Time_Sries dataset to aviod long running time
ID = Time_Series['id']
ID.drop_duplicates(inplace=True) # Keep the unique value 30490

Time_Series_one_ID = ID[:7624].values.tolist()
Time_Series_two_ID = ID[7624:15247].values.tolist()
Time_Series_three_ID = ID[15247:22870].values.tolist()
Time_Series_four_ID = ID[22870:].values.tolist()

### 5: Set the id to index, faster filtering time
Time_Series = Time_Series.set_index('id')

### 6: Extract by ID
Time_Series_one = Time_Series[Time_Series.index.isin(Time_Series_one_ID)]
Time_Series_two = Time_Series[Time_Series.index.isin(Time_Series_two_ID)]
Time_Series_three = Time_Series[Time_Series.index.isin(Time_Series_three_ID)]
Time_Series_four = Time_Series[Time_Series.index.isin(Time_Series_four_ID)]

### 7: Convert the ID back
Time_Series_one.reset_index(inplace=True)
Time_Series_two.reset_index(inplace=True)
Time_Series_three.reset_index(inplace=True)
Time_Series_four.reset_index(inplace=True)

### 8: Save the dataset
Time_Series_one.to_csv('/content/drive/My Drive/Colab Notebooks/Data set/Time Series/Time_Series_one.csv',index=False)
Time_Series_two.to_csv('/content/drive/My Drive/Colab Notebooks/Data set/Time Series/Time_Series_two.csv',index=False)
Time_Series_three.to_csv('/content/drive/My Drive/Colab Notebooks/Data set/Time Series/Time_Series_three.csv',index=False)
Time_Series_four.to_csv('/content/drive/My Drive/Colab Notebooks/Data set/Time Series/Time_Series_four.csv',index=False)

## Prediction <a id="6.2"></a>
> - Because the newly created Time_Series File is a [Panel Data](https://en.wikipedia.org/wiki/Panel_data#:~:text=In%20statistics%20and%20econometrics%2C%20panel,the%20same%20firms%20or%20individuals.); therefore we made the prediction for each unique ID by for loop
> - The running time for each Time_Series file is roughly 5 hours, so the total running time for four files is close to 20 hours
> - In here, I will only make prediction for 5 unique ids for the next 56 days (28 for validation & 28 for evaluation) to save time

The Final Submission score by using FB Prophet is `0.72804`. 

In [ ]:
Time_Series_one = pd.read_csv('../input/sample-prediction-dataset/Hao_Time_Series.csv',parse_dates=['date'])

In [ ]:
### 1: Delete the index from previous save 
Time_Series_one.drop(labels='Unnamed: 0',axis=1,inplace=True)

### 2: Set index as id to retrieve specific id related info
Time_Series_one.set_index('id',inplace=True)
Time_Series_one.index.name = None

### 3: Get dataset ready for the format required by prophet
Time_Series_one.rename(columns={'Unit_Sales':'y','date':'ds'},inplace=True)

### 4: Create a list contains each unique ID for prediction use
Time_id = Time_Series_one.index
Time_id = Time_id.drop_duplicates()

In [ ]:
Time_Series_one.head()

In [ ]:
### 5: Create a mini dataset for demonstration
Time_Series_Mini = Time_Series_one.loc[['HOBBIES_1_001_CA_1_validation','HOBBIES_1_002_CA_1_validation',
                   'HOBBIES_1_003_CA_1_validation','HOBBIES_1_004_CA_1_validation','HOBBIES_1_005_CA_1_validation'],:]

### 6: Save the Time_Series_Mini_ID for prediction result
Time_Series_Mini_ID = Time_Series_Mini.index.drop_duplicates().tolist()

In [ ]:
### 7: Run the prediction by FB Prophet
holder = []

for id in Time_Series_Mini_ID: ### Run prediction on each Unique ID by for loop
  data = Time_Series_Mini.loc[id,:]
  prophet = fbprophet.Prophet(daily_seasonality=True)
  prophet = prophet.fit(data)
# We don't care the past unit sales, but you definitely keep these entries to exam the performance
  Future_Data = prophet.make_future_dataframe(periods=56,include_history=False) 
  Prediction = prophet.predict(Future_Data)
  Prediction = Prediction.loc[:,'yhat'] # yhat is the predicted result, so keep only this column
  holder.append(Prediction)

In [ ]:
### 8: Turn the result to dataframe and here we have the 56 days unit sales prediction
Result = pd.DataFrame(holder,index=Time_Series_Mini_ID)

In [ ]:
### 9: Create an empty holder to get xxx_xx_evaluation ID
Time_Series_Mini_ID_evaluation = []

# 9-1: Take out 'validation' and add 'evaluation'
for iden in Time_Series_Mini_ID:
    new_id = iden[:-10] + 'evaluation'
    Time_Series_Mini_ID_evaluation.append(new_id)

# 9-2: Create the full id list
for i in Time_Series_Mini_ID_evaluation:
    Time_Series_Mini_ID.append(i)

### 10: Create the column name list
Result_Column_List = ["F" + str(i) for i in range(1,29)]

### 11: Seperate the Result dataset to required submission format
Result_validation = Result.iloc[:,0:28]
Result_evaluation = Result.iloc[:,28:56]

### 12: Change the column name for both dataset
Result_validation.columns = Result_Column_List
Result_evaluation.columns = Result_Column_List

### 13: Append two dataset
Result = pd.concat([Result_validation,Result_evaluation])

### 14: Change the Result dataset's index to required format
Result.index = Time_Series_Mini_ID

### 15: Reset the index to numerical & rename to `id` 
Result.reset_index(level=0,inplace=True)
Result.rename(columns = {'index':'id'},inplace=True)

In [ ]:
### Final Result for Prediction with FB Prophet - Time Series Mini Dataset
Result

# Prediction Methodology 2 - Time Series With LSTM <a id="7"></a>


## Data Merging <a id="7.1"></a>
> - Since the evaluation data belongs to May & June, we only merge information related to these month


In [ ]:
### 1: Calendar 

# 1-1: Train Calender (d_1 - d_1913)
calendar_train = calendar[(calendar['d'] > 0) & (calendar['d'] < 1914)]
calendar_train_1 = calendar_train[calendar_train['month']==5]
calendar_train_2 = calendar_train[calendar_train['month']==6]
calendar_train = pd.concat((calendar_train_1,calendar_train_2))
# 1-2: Validation Calender (d_1914 - d_1942)
calendar_validation = calendar[(calendar['d']>1913) & (calendar['d'] < 1941)]
# 1-3: Eveluation Calender (d_1942 - d_1969)
calendar_eveluation = calendar[calendar['d']>1941]

### 2: Seperate the sell_price for train, validation, and evaluation

### 2-1: Get sell_price for train data 
price_train_week = calendar_train['wm_yr_wk'].array
sell_price_train = sell_price[sell_price['wm_yr_wk'].isin(price_train_week)]
### 2-2: Get sell_price for validation data 
price_validation_week = calendar_validation['wm_yr_wk'].array
sell_price_validation = sell_price[sell_price['wm_yr_wk'].isin(price_validation_week)]
### 2-3: Get sell_price for eveluation data
price_eve_week = calendar_eveluation['wm_yr_wk'].array
sell_price_eveluation = sell_price[sell_price['wm_yr_wk'].isin(price_eve_week)]

### 3: Get the merged Validation & Evaluation dataset
# 3-1: Validation dataset
Validation_Data = sell_price_validation.merge(right=calendar_validation,how='left',on='wm_yr_wk',copy=False)
Validation_Data['id'] = Validation_Data['item_id'] + '_' + Validation_Data['store_id'] + '_validation'  
Validation_Data.drop(labels=['store_id','item_id'],axis=1,inplace=True)
# 3-2: Evaluation dataset
Eveluation_Data = sell_price_eveluation.merge(right=calendar_eveluation,how='left',on='wm_yr_wk',copy=False)
Eveluation_Data['id'] = Eveluation_Data['item_id'] + '_' + Eveluation_Data['store_id'] + '_evaluation'  
Eveluation_Data.drop(labels=['store_id','item_id'],axis=1,inplace=True)

### 4: Get the Merged Train Dataset
train_calendar_price = sell_price_train.merge(right=calendar_train,on='wm_yr_wk',how='left',copy=False)
train_dataset = sales_train_validation.merge(right=train_calendar_price,on=['item_id','store_id','d'],how='inner',copy=False)
train_dataset.drop(labels=['item_id','store_id','cat_id','state_id'],axis=1,inplace=True)

In [ ]:
### Until This point the merged and ready-to-use train, validation, and evaluation dataset are ready
### But to use it in deep learning, we need do some feature engineering
#__________________________________________________________________________________________________________________________#

### 1: Train Dataset

ID = train_dataset['id']
train_dataset.drop(labels=['d','wm_yr_wk','date','event_type_2'],axis=1,inplace=True)
train_dataset.drop(labels=['month','year'],axis=1,inplace=True)

# 1-1: Get New Variable Category based on ID

train_dataset['category'] = train_dataset['id'].str[:9]
train_dataset.loc[train_dataset['category']=='HOUSEHOLD','category'] = 0
train_dataset.loc[train_dataset['category']=='HOBBIES_1','category'] = 1
train_dataset.loc[train_dataset['category']=='HOBBIES_2','category'] = 2
train_dataset.loc[train_dataset['category'] == 'FOODS_1_0','category'] = 2
train_dataset.loc[train_dataset['category'] == 'FOODS_1_1','category'] = 2
train_dataset.loc[train_dataset['category'] == 'FOODS_3_4','category'] = 2
train_dataset.loc[train_dataset['category'] == 'FOODS_3_2','category'] = 2
train_dataset.loc[train_dataset['category'] == 'FOODS_3_3','category'] = 2
train_dataset.loc[train_dataset['category'] == 'FOODS_3_7','category'] = 2
train_dataset.loc[train_dataset['category'] == 'FOODS_3_1','category'] = 2
train_dataset.loc[train_dataset['category'] == 'FOODS_3_6','category'] = 2
train_dataset.loc[train_dataset['category'] == 'FOODS_3_0','category'] = 2
train_dataset.loc[train_dataset['category'] == 'FOODS_3_5','category'] = 2
train_dataset.loc[train_dataset['category'] == 'FOODS_2_3','category'] = 2
train_dataset.loc[train_dataset['category'] == 'FOODS_2_1','category'] = 2
train_dataset.loc[train_dataset['category'] == 'FOODS_2_0','category'] = 2
train_dataset.loc[train_dataset['category'] == 'FOODS_2_2','category'] = 2
train_dataset.loc[train_dataset['category'] == 'FOODS_3_8','category'] = 2
train_dataset.loc[train_dataset['category'] == 'FOODS_1_2','category'] = 2

# 1-2: Dummy Encoding weekday & category

weekday = pd.get_dummies(train_dataset['weekday'])
weekday.columns = ['weekday_0','weekday_1','weekday_2','weekday_3','weekday_4','weekday_5','weekday_6']
train_dataset = pd.concat([train_dataset,weekday],axis=1)
train_dataset.drop(labels='weekday',axis=1,inplace=True)

category = pd.get_dummies(train_dataset['category'])
category.columns = ['category_0','category_1','category_2']
train_dataset = pd.concat([train_dataset,category],axis=1)
train_dataset.drop(labels='category',axis=1,inplace=True)

# 1-3: Process the Continouos Variable sell_price and divided by largest value of sell_price
train_dataset['sell_price'] = train_dataset['sell_price'] / max(train_dataset['sell_price'])

# 1-4:  Sort the Dataframe to match with the id sequence in Train & make sure the d is ascending
train_ready['d'] = train_dataset.d
ID_v = train_ready['id'] 
sortIndex = dict(zip(ID_v,train_ready.index))
train_ready['SortIndex'] = train_ready['id'].map(sortIndex)
train_ready.sort_values(by=['d','SortIndex'],ascending=[True,True],inplace=True)
train_ready.drop(labels='SortIndex',axis=1,inplace=True)

#__________________________________________________________________________________________________________________________#

### 2: Validation Dataset

ID_V = Validation_Data['id']
Validation_Data.drop(labels=['store_id','item_id','date','event_type_2'],axis=1,inplace=True)
Validation_Data.drop(labels=['month','year'],axis=1,inplace=True)

# 2-1: Get New Variable Category based on ID

Validation_Data['category'] = Validation_Data['id'].str[:9]
Validation_Data.loc[Validation_Data['category']=='HOUSEHOLD','category'] = 0
Validation_Data.loc[Validation_Data['category']=='HOBBIES_1','category'] = 1
Validation_Data.loc[Validation_Data['category']=='HOBBIES_2','category'] = 2
Validation_Data.loc[Validation_Data['category'] == 'FOODS_1_0','category'] = 2
Validation_Data.loc[Validation_Data['category'] == 'FOODS_1_1','category'] = 2
Validation_Data.loc[Validation_Data['category'] == 'FOODS_3_4','category'] = 2
Validation_Data.loc[Validation_Data['category'] == 'FOODS_3_2','category'] = 2
Validation_Data.loc[Validation_Data['category'] == 'FOODS_3_3','category'] = 2
Validation_Data.loc[Validation_Data['category'] == 'FOODS_3_7','category'] = 2
Validation_Data.loc[Validation_Data['category'] == 'FOODS_3_1','category'] = 2
Validation_Data.loc[Validation_Data['category'] == 'FOODS_3_6','category'] = 2
Validation_Data.loc[Validation_Data['category'] == 'FOODS_3_0','category'] = 2
Validation_Data.loc[Validation_Data['category'] == 'FOODS_3_5','category'] = 2
Validation_Data.loc[Validation_Data['category'] == 'FOODS_2_3','category'] = 2
Validation_Data.loc[Validation_Data['category'] == 'FOODS_2_1','category'] = 2
Validation_Data.loc[Validation_Data['category'] == 'FOODS_2_0','category'] = 2
Validation_Data.loc[Validation_Data['category'] == 'FOODS_2_2','category'] = 2
Validation_Data.loc[Validation_Data['category'] == 'FOODS_3_8','category'] = 2
Validation_Data.loc[Validation_Data['category'] == 'FOODS_1_2','category'] = 2

# 2-2: Dummy Encoding weekday & category
weekday = pd.get_dummies(Validation_Data['weekday'])
weekday.columns = ['weekday_0','weekday_1','weekday_2','weekday_3','weekday_4','weekday_5','weekday_6']
Validation_Data = pd.concat([Validation_Data,weekday],axis=1)
Validation_Data.drop(labels='weekday',axis=1,inplace=True)

category = pd.get_dummies(Validation_Data['category'])
category.columns = ['category_0','category_1','category_2']
Validation_Data = pd.concat([Validation_Data,category],axis=1)
Validation_Data.drop(labels='category',axis=1,inplace=True)

# 2-3: Process the Continouos Variable sell_price and divided by largest value of sell_price
Validation_Data['sell_price'] = Validation_Data['sell_price'] / max(Validation_Data['sell_price'])

# 2-4:  Sort the Dataframe to match with the id sequence in Train & make sure the d is ascending
validation_ready['d'] = Validation_Data.d
ID_v = validation_ready['id'] 
sortIndex = dict(zip(ID_v,validation_ready.index))
validation_ready['SortIndex'] = validation_ready['id'].map(sortIndex)
validation_ready.sort_values(by=['d','SortIndex'],ascending=[True,True],inplace=True)
validation_ready.drop(labels='SortIndex',axis=1,inplace=True)

#__________________________________________________________________________________________________________________________#

### 3: Eveluation Dataset

ID_V = Eveluation_Data['id']
Eveluation_Data.drop(labels=['store_id','item_id','date','event_type_2'],axis=1,inplace=True)
Eveluation_Data.drop(labels=['month','year'],axis=1,inplace=True)

# 3-1: Get New Variable Category based on ID

Eveluation_Data['category'] = Eveluation_Data['id'].str[:9]
Eveluation_Data.loc[Eveluation_Data['category']=='HOUSEHOLD','category'] = 0
Eveluation_Data.loc[Eveluation_Data['category']=='HOBBIES_1','category'] = 1
Eveluation_Data.loc[Eveluation_Data['category']=='HOBBIES_2','category'] = 2
Eveluation_Data.loc[Eveluation_Data['category'] == 'FOODS_1_0','category'] = 2
Eveluation_Data.loc[Eveluation_Data['category'] == 'FOODS_1_1','category'] = 2
Eveluation_Data.loc[Eveluation_Data['category'] == 'FOODS_3_4','category'] = 2
Eveluation_Data.loc[Eveluation_Data['category'] == 'FOODS_3_2','category'] = 2
Eveluation_Data.loc[Eveluation_Data['category'] == 'FOODS_3_3','category'] = 2
Eveluation_Data.loc[Eveluation_Data['category'] == 'FOODS_3_7','category'] = 2
Eveluation_Data.loc[Eveluation_Data['category'] == 'FOODS_3_1','category'] = 2
Eveluation_Data.loc[Eveluation_Data['category'] == 'FOODS_3_6','category'] = 2
Eveluation_Data.loc[Eveluation_Data['category'] == 'FOODS_3_0','category'] = 2
Eveluation_Data.loc[Eveluation_Data['category'] == 'FOODS_3_5','category'] = 2
Eveluation_Data.loc[Eveluation_Data['category'] == 'FOODS_2_3','category'] = 2
Eveluation_Data.loc[Eveluation_Data['category'] == 'FOODS_2_1','category'] = 2
Eveluation_Data.loc[Eveluation_Data['category'] == 'FOODS_2_0','category'] = 2
Eveluation_Data.loc[Eveluation_Data['category'] == 'FOODS_2_2','category'] = 2
Eveluation_Data.loc[Eveluation_Data['category'] == 'FOODS_3_8','category'] = 2
Eveluation_Data.loc[Eveluation_Data['category'] == 'FOODS_1_2','category'] = 2

# 3-2: Dummy Encoding weekday & category
weekday = pd.get_dummies(Eveluation_Data['weekday'])
weekday.columns = ['weekday_0','weekday_1','weekday_2','weekday_3','weekday_4','weekday_5','weekday_6']
Eveluation_Data = pd.concat([Eveluation_Data,weekday],axis=1)
Eveluation_Data.drop(labels='weekday',axis=1,inplace=True)

category = pd.get_dummies(Eveluation_Data['category'])
category.columns = ['category_0','category_1','category_2']
Eveluation_Data = pd.concat([Eveluation_Data,category],axis=1)
Eveluation_Data.drop(labels='category',axis=1,inplace=True)

# 3-3: Process the Continouos Variable sell_price and divided by largest value of sell_price
Eveluation_Data['sell_price'] = Eveluation_Data['sell_price'] / max(Eveluation_Data['sell_price'])

# 2-4:  Sort the Dataframe to match with the id sequence in Train & make sure the d is ascending
Evaluation_ready['d'] = Eveluation_Data.d
ID_v = Evaluation_ready['id'] 
sortIndex = dict(zip(ID_v,Evaluation_ready.index))
Evaluation_ready['SortIndex'] = Evaluation_ready['id'].map(sortIndex)
Evaluation_ready.sort_values(by=['d','SortIndex'],ascending=[True,True],inplace=True)
Evaluation_ready.drop(labels='SortIndex',axis=1,inplace=True)

In [ ]:
### Take a Look of train dataset. Since the validation & evaluation has same column as validation 
### (Except Unit_Sales is not in both dataset)
train_ready.head()

## Prediction <a id="7.2"></a>

> - For Prediction, I used the LSTM method. However, This is the first time I use deep learning; therefore, the result is horrific
> - First, I used the dense layer to create a baseline, the RMSE for validation split is approximatly 3.2; Then I used the LSTM layer to create the Final model, but the performance is even worse. 
> - Any suggestion or feedback for this section will be appreciated


In [ ]:
### Data preparation for LSTM, turn 2D array to 3D as input
x_train = train_ready.drop(labels='Unit_Sales',axis=1)
y_train = train_ready['Unit_Sales']

x_train_float = x_train.to_numpy()
y_train_float = y_train.to_numpy()

x_train_lstm = x_train_float.reshape((x_train_float.shape[0],1,x_train_float.shape[1])) 
y_train_lstm = y_train_float


print(x_train_float.shape) 
print(y_train_float.shape)
print(x_train_lstm.shape) 
print(y_train_lstm.shape)

In [ ]:
### Use rmse as validation metrics & Early Stopping to evaluate the model performance
rmse = RootMeanSquaredError()
monitor = EarlyStopping(patience=3,monitor='val_root_mean_squared_error') 

In [ ]:
### Build First Model as Dense layer - set the baseline
model = Sequential()
model.add(Dense(64,activation='relu',input_shape = (15,)))
model.add(BatchNormalization())
model.add(Dense(64,activation='relu'))
model.add(BatchNormalization())
model.add(Dense(64,activation='relu'))
model.add(BatchNormalization())
model.add(Dense(64,activation='relu'))
model.add(BatchNormalization())
model.add(Dense(1))
model.compile(optimizer='Adam',loss='mean_squared_error',metrics=[rmse])
model.fit(x= x_train_float,y= y_train_float,epochs=20,validation_split=0.2,callbacks=[monitor],batch_size=600)

train_loss = model.history.history['loss']
train_rmse = model.history.history['root_mean_squared_error']
validation_loss = model.history.history['val_loss']
validation_rmse = model.history.history['val_root_mean_squared_error']

### Plot Loss
plt.plot(range(1,8),train_loss,'orange',label='train_loss')
plt.plot(range(1,8),validation_loss,'green',label='validation_loss')
plt.legend()
plt.title('Trian & Validation loss')
plt.xlabel('Epoch')
plt.ylabel('loss')
plt.show()

### Plot rmse
plt.plot(range(1,8),train_rmse,'green',label='Train rmse')
plt.plot(range(1,8),validation_rmse,'orange',label='Validation rmse')
plt.legend()
plt.title('Trian & Validation rmse')
plt.xlabel('Epoch')
plt.ylabel('rmse')
plt.show()

<img src="http://drive.google.com/uc?export=view&id=16oCFXeI-xxOzKOaqReuzOuPL9uTdGmuK" width=400px>
<img src="http://drive.google.com/uc?export=view&id=1p3dNP9174oYtTJYsaSt4VHlA9t1fJDrr" width=400px>

In [ ]:
K.clear_session()
### Build Second Model as LSTM layer

model_lstm = Sequential()
model_lstm.add(LSTM(64,input_shape=(x_train_lstm.shape[1],x_train_lstm.shape[2]),return_sequences=True))
model_lstm.add(LSTM(64,return_sequences=True))
model_lstm.add(LSTM(64,return_sequences=True))
model_lstm.add(LSTM(64,return_sequences=True))
model_lstm.add(LSTM(64,return_sequences=True))
model_lstm.add(LSTM(64,return_sequences=True))
model_lstm.add(LSTM(64,return_sequences=True))
model_lstm.add(LSTM(64,return_sequences=True))
model_lstm.add(LSTM(64,return_sequences=True))
model_lstm.add(LSTM(64,return_sequences=True))
model_lstm.add(LSTM(64,return_sequences=True))
model_lstm.add(LSTM(64))
model_lstm.add(Dense(1))
model_lstm.compile(optimizer='rmsprop',loss='mean_squared_error',metrics=[rmse])
model_lstm.fit(x= x_train_lstm,y= y_train_lstm,epochs=20,validation_split=0.2,callbacks=[monitor],batch_size=200)

train_loss = model_lstm.history.history['loss']
train_rmse = model_lstm.history.history['root_mean_squared_error']
validation_loss = model_lstm.history.history['val_loss']
validation_rmse = model_lstm.history.history['val_root_mean_squared_error']

### Plot Loss
plt.plot(range(1,16),train_loss,'orange',label='train_loss')
plt.plot(range(1,16),validation_loss,'green',label='validation_loss')
plt.legend()
plt.title('Trian & Validation loss')
plt.xlabel('Epoch')
plt.ylabel('loss')
plt.show()

### Plot rmse
plt.plot(range(1,16),train_rmse,'green',label='Train rmse')
plt.plot(range(1,16),validation_rmse,'orange',label='Validation rmse')
plt.legend()
plt.title('Trian & Validation rmse')
plt.xlabel('Epoch')
plt.ylabel('rmse')
plt.show()

<img src="http://drive.google.com/uc?export=view&id=1fV2CwkwKo_iAKMZ6Ezp1vWMLRb-5mGOC" width=400px>
<img src="http://drive.google.com/uc?export=view&id=1hzct4qNzAG6QUWhOKG3WKc5BvPMqlpdz" width=400px>